In [203]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import csv 
import re
from bs4 import BeautifulSoup
import unicodedata
from selenium.webdriver.common.by import By
import pandas as pd


In [206]:
keyword = ['Myeongdong','명동','明洞']

email = '인스타그램_아이디'
password = '인스타그램_비밀번호'

In [207]:
# selenium 이용해서 드라이버 지정하고 url 불러오기
# keyword 를 포함한 url로 쓰기
driver = webdriver.Chrome(r"C:\Users\USER\Downloads\chromedriver.exe")

# 인스타그램 접속하기
insta_url = 'https://www.instargram.com' 
driver.get(insta_url)
#time.sleep(2)
time.sleep(3)

C:\Users\USER\AppData\Local\Temp/ipykernel_6744/314291897.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\USER\Downloads\chromedriver.exe")


In [208]:
# 예제 5-3 인스타계정으로 로그인하기
#email = '인스타계정입력해주세요'   ### 계정 정보 수정 필요
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
#input_id.clear()
input_id.clear()
input_id.send_keys(email)

#password = '인스타비번입력해주세요' ### 비번 정보 수정 필요
time.sleep(1)
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(3)
# 로그인 버튼 클릭 click
login = driver.find_element_by_css_selector("button.sqdOP.L3NKy.y3zKF")
login.click()


C:\Users\USER\AppData\Local\Temp/ipykernel_6744/2901931748.py:3: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
C:\Users\USER\AppData\Local\Temp/ipykernel_6744/2901931748.py:10: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
C:\Users\USER\AppData\Local\Temp/ipykernel_6744/2901931748.py:16: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  login = driver.find_element_by_css_selector("button.sqdOP.L3NKy.y3zKF")


In [209]:
#time.sleep(2)
## 로그인 시 팝업 창 제거
# 로그인 저장 팝업 제거
later_button = driver.find_elements_by_class_name("sqdOP")
later_button[0].click()
time.sleep(1)
# 알림창 제거
alert = driver.find_elements_by_css_selector("button.aOOlW")
alertLater = alert[1]
alertLater.click()

C:\Users\USER\AppData\Local\Temp/ipykernel_6744/3559789725.py:4: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  later_button = driver.find_elements_by_class_name("sqdOP")
C:\Users\USER\AppData\Local\Temp/ipykernel_6744/3559789725.py:8: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  alert = driver.find_elements_by_css_selector("button.aOOlW")


In [210]:
def insta_searching(word) :
    url = insta_url + "/explore/tags/" + word
    return url

url = insta_searching(keyword[0]) # Myeong Dong
driver.get(url)
time.sleep(1)

In [211]:
# 예제 5-6 HTML에서 첫번째 게시글 찾아 클릭하기
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)
    
select_first(driver)

C:\Users\USER\AppData\Local\Temp/ipykernel_6744/556389458.py:3: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  first = driver.find_element_by_css_selector("div._9AhH0")


In [214]:
# 예제 5-7 게시글 정보 가져오기
def get_content(driver):
    # ① 현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # ② 본문 내용 가져오기
    try:
        # 본문에 사용자 id가 포함
        user_id_len = len(soup.select('a.sqdOP.yWX7d._8A5w5.ZIAjV')[0].text)
        
        # 본문에서 사용자 id 제거
        content = soup.select('div.C4VMK')[0].text[user_id_len:]
        content = unicodedata.normalize('NFC', content) 
        #print(" ".join(re.sub(r'#[^\s#,\\]+' ,"",content).split()))
    except:
        content = ' '
    # ③ 본문 내용에서 해시태그 가져오기(정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content.replace('수정됨',''))
    # ④ 작성일자 정보 가져오기
    #date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    date = soup.select('time._1o9PC')[0]['datetime'][:10]
    # ⑤ 좋아요 수 가져오기
    try:
        #like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
        like = int(soup.select('div._7UhW9.xLCgt.qyrsm.KV-D4.fDxYl.T0kll')[0].text[4:-1].replace(',',''))
    except:
        like = 0
    # ⑥ 위치정보 가져오기
    try: 
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
    # ⑦ 수집한 정보 저장하기
    #data = [" ".join(re.sub(r'#[^\s#,\\]+' ,"",content).split()), date, like, place, tags]
    data = [date, like, place, tags]
    return data

get_content(driver)

['2022-04-30',
 758,
 '',
 ['#韓国',
  '#渡韓',
  '#渡韓記録',
  '#韓国旅行',
  '#韓国女子旅',
  '#韓国女子',
  '#ソウル旅行',
  '#韓国一人旅',
  '#韓国情報',
  '#韓国旅行記',
  '#韓国ファッション',
  '#韓国購入品',
  '#韓国ファッション通販',
  '#韓国コーデ',
  '#韓国トレンド',
  '#オルチャンコーデ',
  '#オルチャンファッション',
  '#韓国行きたい',
  '#カロスキル',
  '#明洞',
  '#東大門',
  '#南大門',
  '#江南',
  '#弘大',
  '#梨大1주']]

In [213]:
# 예제 5-8 다음 게시글 열기
def move_next(driver):
    #/html/body/div[6]/div[2]/div/div/button/div/span/svg
    #right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    #right = driver.find_element_by_css_selector ('button.wpO6b')
    right = driver.find_element(by = By.XPATH, value = '/html/body/div[6]/div[2]/div/div[2]')
    #right = driver.find_element(by = By.XPATH, value = '/html/body/div[6]/div[2]/div/div/button/div/span')
    right.click()
    time.sleep(3)

move_next(driver)

In [ ]:
results = []
target = 1000      # 크롤링할 게시글 수
for i in range(target):
    # 게시글 수집에 오류 발생시(네트워크 문제 등의 이유로)  2초 대기 후, 다음 게시글로 넘어가도록 try, except 구문 활용
    try:
        data = get_content(driver)    # 게시글 정보 가져오기
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)
    if i%50 == 0 :
        time.sleep(3)

In [180]:
df = pd.DataFrame(results)
df.columns= ['게시물_날짜','좋아요_수','위치','해시태그']
df = df.sort_values(['게시물_날짜','좋아요_수'],ascending=[True,False])#

In [202]:
df

,게시물_날짜,좋아요_수,위치,해시태그
153,2017-11-16,6,Myeongdong Cathedral,"[#Korea, #seoul, #myeongdong, #教会, #明洞]"
182,2021-09-24,0,오유우,[]
181,2021-10-14,0,"Al,thing and kitchen",[#Repost]
180,2021-11-03,0,오유우,[]
6,2021-12-10,1212,신세계명동본점,"[#서울, #명동, #신세계본점, #크리스마스, #미리크리스마스, #메리크리스마스,..."
...,...,...,...,...
24,2022-05-10,0,L'Escape Hotel,[#레스케이프호텔@lescape_hotel]
25,2022-05-10,0,,"[#likeforlikes, #instaovalle, #instaoutdoors, ..."
9,2022-05-11,16,대한민국,[#SeoulMate]
7,2022-05-11,0,L'Escape Hotel,"[#벨에포크, #파리여행, #공주놀이호텔, #취향저격, #레스케이프호텔@lescap..."
